In [1]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

from pynq.lib.arduino import Arduino_RA8875
test = Arduino_RA8875(base.ARDUINO)

test.init()
test.display()
test.gpio()
test.pwm()


3

In [2]:
def _convert_color(color):
    """Convert a 24-bit color to 16-bit.

    The input `color` is assumed to be a 3-component list [R,G,B], each with
    8 bits for color level.

    This method will translate that list of colors into a 16-bit number,
    with first 5 bits indicating R component,
    last 5 bits indicating B component, and remaining
    6 bits in the middle indicating G component.
    i.e., 16-bit color -> (5 bits, 6 bits, 5 bits) -> (R,G,B).

    """
    for i in color:
        if i not in range(256):
            raise ValueError("Valid color value for R, G, B is 0 - 255.")
    red, green, blue = color
    return ((blue & 0xF8) << 8) | ((green & 0xFC) << 3) | ((red & 0xF8) >> 3)

In [16]:
from PIL import Image
from numpy import array
img = Image.open('data/Yogananda.jpg')
img.resize((240,240), Image.ANTIALIAS)
img_array = array(img)
img.close()

file_size = 240*240*2
buf0 = test.buf_manager.cma_alloc(file_size, data_type="uint8_t")
buf1 = test.buf_manager.cma_get_buffer(buf0, file_size)
phy_addr = test.buf_manager.cma_get_phy_addr(buf0)
index = 0
image_save = open("yoga.txt", "w")
for j in range(240):
    for i in range(240):
        red, green, blue = img_array[i][j]
#         pixel = ((blue & 0xF8) << 8) | ((green & 0xFC) << 3) | ((red & 0xF8) >> 3)
        pixel = ((blue & 0xF8) >> 3) | ((green & 0xFC) << 3) | ((red & 0xF8) << 8)
        image_save.write(hex(pixel) + "\n")
        index = 2 * ((240 - i - 1) * 240 + j)
        buf1[index] = bytes([pixel & 0xFF])
        buf1[index + 1] = bytes([(pixel & 0xFF00) >> 8])
image_save.close()
test.microblaze.write_mailbox(0, 0)
test.microblaze.write_mailbox(4, 0)
test.microblaze.write_mailbox(8, phy_addr)
test.microblaze.write_mailbox(12, 240)
test.microblaze.write_mailbox(16, 240)
test.microblaze.write_blocking_command(0x8)

In [38]:
from PIL import Image
from numpy import array
img = Image.open('data/logo_small.png')
width, height = img.size
img.resize((width,height), Image.ANTIALIAS)
img_array = array(img)
img.close()

file_size = width*height*2
buf0 = test.buf_manager.cma_alloc(file_size, data_type="uint8_t")
buf1 = test.buf_manager.cma_get_buffer(buf0, file_size)
phy_addr = test.buf_manager.cma_get_phy_addr(buf0)
index = 0
for j in range(width):
    for i in range(height):
        red, green, blue = img_array[i][j]
#         pixel = ((blue & 0xF8) << 8) | ((green & 0xFC) << 3) | ((red & 0xF8) >> 3) #original
        pixel = ((blue & 0xF8) >> 3) | ((green & 0xFC) << 3) | ((red & 0xF8) << 8) 
#         pixel_orig = ((blue & 0xF8) >> 3) | ((green & 0xFC) << 3) | ((red & 0xF8) << 8)
#         pixel = int('{:016b}'.format(pixel_orig)[::-1])
        index = 2 * ((height - i - 1) * width + j)
        buf1[index] = bytes([pixel & 0xFF])
        buf1[index + 1] = bytes([(pixel & 0xFF00) >> 8])
test.microblaze.write_mailbox(0, 0)
test.microblaze.write_mailbox(4, 0)
test.microblaze.write_mailbox(8, phy_addr)
test.microblaze.write_mailbox(12, height)
test.microblaze.write_mailbox(16, width)
test.microblaze.write_blocking_command(0x8)

In [37]:
binary = 3855
flipped_binary = '{:16b}'.format(binary)[::-1]
print(bin(binary))
print(bin(int(flipped_binary)))

0b111100001111
0b1100111011110000100110111101101010111


In [17]:
from PIL import Image
from numpy import array
img = Image.open('data/adafruit_lcd18.jpg')
width, height = img.size
img.resize((width,height), Image.ANTIALIAS)
img_array = array(img)
img.close()

file_size = width*height*2
buf0 = test.buf_manager.cma_alloc(file_size, data_type="uint8_t")
buf1 = test.buf_manager.cma_get_buffer(buf0, file_size)
phy_addr = test.buf_manager.cma_get_phy_addr(buf0)
index = 0
for j in range(width):
    for i in range(height):
        red, green, blue = img_array[i][j]
        pixel = ((blue & 0xF8) << 8) | ((green & 0xFC) << 3) | ((red & 0xF8) >> 3)
        index = 2 * ((height - i - 1) * width + j)
        buf1[index] = bytes([pixel & 0xFF])
        buf1[index + 1] = bytes([(pixel & 0xFF00) >> 8])
test.microblaze.write_mailbox(0, 0)
test.microblaze.write_mailbox(4, 0)
test.microblaze.write_mailbox(8, phy_addr)
test.microblaze.write_mailbox(12, height)
test.microblaze.write_mailbox(16, width)
test.microblaze.write_blocking_command(0x8)

In [6]:
output_list = test.draw_yoga(0,0)
for vals in output_list:
    print(hex(vals))

0x0
0x0
0x168a0000
0xf0
0xf0


In [1]:
#checking values stored in DDR v1
x_start = 0
y_start = 0
file_size = len(test.yoga)*2
buf0 = test.buf_manager.cma_alloc(file_size, data_type="uint8_t")
buf1 = test.buf_manager.cma_get_buffer(buf0, file_size)
phy_addr = test.buf_manager.cma_get_phy_addr(buf0)
index = 0
for i in range(0, 5):
    pixel = test.yoga[i]
    bin_val = "{0:b}".format(pixel)
    hex_val = '{:x}'.format(pixel)
#     print (bin_val)
    bv_1 = bin_val[:8]
    bv_2 = bin_val[8:]
#     print (bv_1)
#     print (bv_2)
    buf1[index] = bytes([int(bv_1, 2)])
    buf1[index+1]= bytes([int(bv_2, 2)])
    print(hex_val)
    print(bin_val)
    print('bv_1 = ' + bv_1)
    print('bv_2 = ' + bv_2)
    print(buf1[index])
    print(buf1[index+1])
    index += 2
test.microblaze.write_mailbox(0, x_start)
test.microblaze.write_mailbox(4, y_start)
test.microblaze.write_mailbox(8, phy_addr)
test.microblaze.write_mailbox(12, 240)
test.microblaze.write_mailbox(16, 240)
test.microblaze.write_blocking_command(0x8)

NameError: name 'test' is not defined

In [4]:
#checking values stored in DDR v2
x_start = 0
y_start = 0
file_size = len(test.yoga)*2
buf0 = test.buf_manager.cma_alloc(file_size, data_type="uint8_t")
buf1 = test.buf_manager.cma_get_buffer(buf0, file_size)
phy_addr = test.buf_manager.cma_get_phy_addr(buf0)
index = 0
for i in range(0, 5):
    pixel = test.yoga[i]
    buf1[index] = bytes([(pixel&0xFF00)>>8])
    buf1[index+1] = bytes([pixel&0x00FF])
    
    print('pixel = ' + str(hex(pixel)) + ', ' + str(pixel))
    print(buf1[index])
    print(buf1[index+1])
    index += 2
test.microblaze.write_mailbox(0, x_start)
test.microblaze.write_mailbox(4, y_start)
test.microblaze.write_mailbox(8, phy_addr)
test.microblaze.write_mailbox(12, 240)
test.microblaze.write_mailbox(16, 240)
test.microblaze.write_blocking_command(0x8)
output_list = test.microblaze.read_mailbox(0,5)
for vals in output_list:
    print(hex(vals) + ', ' + str(vals) + ', ' + bin(vals))

pixel = 0xf363, 62307
b'\xf3'
b'c'
pixel = 0xb25b, 45659
b'\xb2'
b'['
pixel = 0xb35b, 45915
b'\xb3'
b'['
pixel = 0xf45b, 62555
b'\xf4'
b'['
pixel = 0xd35b, 54107
b'\xd3'
b'['
0xf363, 62307, 0b1111001101100011
0xb25b, 45659, 0b1011001001011011
0xb35b, 45915, 0b1011001101011011
0xf45b, 62555, 0b1111010001011011
0xd35b, 54107, 0b1101001101011011


In [20]:
#testing basic fill color
import time

color_table = [0x0000, 0x001F, 0xF800, 0x07E0, 0x07FF, 0xF81F, 0xFFE0, 0xFFFF]
color_index = ['black', 'blue', 'red', 'green', 'cyan', 'magenta', 'yellow', 'white']
for index, color in enumerate(color_table):    
    return_val = test.fill_color(color)
    print('Color: '+ color_index[index])
    print('Decimal = ' + str(return_val))
    print('Hex = ' + hex(return_val))
    print('Bin = ' + bin(return_val))
    print(' ')
    time.sleep(3)

Color: black
Decimal = 0
Hex = 0x0
Bin = 0b0
 
Color: blue
Decimal = 31
Hex = 0x1f
Bin = 0b11111
 
Color: red
Decimal = 63488
Hex = 0xf800
Bin = 0b1111100000000000
 
Color: green
Decimal = 2016
Hex = 0x7e0
Bin = 0b11111100000
 
Color: cyan
Decimal = 2047
Hex = 0x7ff
Bin = 0b11111111111
 
Color: magenta
Decimal = 63519
Hex = 0xf81f
Bin = 0b1111100000011111
 
Color: yellow
Decimal = 65504
Hex = 0xffe0
Bin = 0b1111111111100000
 
Color: white
Decimal = 65535
Hex = 0xffff
Bin = 0b1111111111111111
 


In [17]:
#Testing custom fill colors
return_val = test.fill_color(0xe0d6)
print('Decimal = ' + str(return_val))
print('Hex = ' + hex(return_val))
print('Bin = ' + bin(return_val))

Decimal = 57558
Hex = 0xe0d6
Bin = 0b1110000011010110


In [18]:
#testing drawPixel color
for x in range(0, 240):
    for y in range(0,240):
        test.drawPixel(x, y, 0xe0d6)

In [10]:
#testing fill
test.fill()

1337

In [19]:
value = test.drawPixel(40,40, 0xe0d6)
print(hex(value))

0xe0d6


In [11]:
test.slowFill(0x07E0);

In [4]:
read_in = test.fill()
print (read_in)

1337


In [5]:
index = 0
start_x = 280
start_y = 120
for x in range(start_x, start_x+240):
    for y in range(start_y, start_y+240):
        test.drawPixel(x, y, int(test.yoga[index]))
        index += 1

In [9]:
index = 0
for x in range(0, 254):
    for y in range(0, 536):
        test.drawPixel(y, x, test.spot[index])
        index += 1